In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

# chain.run("Where does Winston live?")
chain.run("Describe Victory Mansions")


'Victory Mansions is a grim and run-down apartment building where Winston Smith, the protagonist of George Orwell\'s novel "1984," resides. The building is described as having a shabby appearance, with broken elevators, faulty plumbing, and dim lighting. The living conditions are poor, with shortages of basic necessities like food. The hallway smells of boiled cabbage and old rag mats, adding to the atmosphere of decay. The oppressive regime of the Party is reflected in the building\'s neglect and the constant surveillance through the telescreens, emphasizing the theme of surveillance and control in the novel. Winston strategically positions himself in an alcove to avoid being seen by the telescreen, allowing him to engage in forbidden activities like reading old, banned books. This act of rebellion highlights the oppressive nature of the Party and the lack of freedom in Victory Mansions. The juxtaposition of Winston\'s small act of defiance within the confines of the decrepit building